In [13]:
from scipy.sparse import csr_matrix
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt

from quantumScarsCheck import *
from quantumScarsPlotting import *
from quantumScarFunctions import *

N = 4 # IDDEN MAKE SURE TO MAKE THIS AN EVEN NUMBER <-----------------------------------------------------------------------------------------------------------------------------
basisList = binNoConsecOnesEfficient(N)
for basis in basisList:
    if basis[0] == '1' and basis[-1] == '1':
        basisList.remove(basis)
print(basisList)
    
basisMap = {bitStr: i for i, bitStr in enumerate(basisList)}
# print(basisMap)
basisLen = len(basisList)
ohms = 1.0

rowBare = []
columnBare = []

rowFactor = []
columnFactor = []

# flip bit hashmap
flipMap = {'0': '1', '1': '0'}

# sigma z op hashmap
sigzMap = {'0': '-1', '1': '1'}

# list of ints for Hamiltonian
numList = []

# -------------------------------
#
# create the bare PXP hamiltonian
#
# -------------------------------
flippedList = []
for i in range(basisLen):

    # add padding so that search doesnt go out of range
    paddedBitStr = basisList[i][-1] + basisList[i] + basisList[i][0]
    #print(paddedBitStr)
    copyBit = list(paddedBitStr)

    # apply the sum of r P_r-1 * sigma_x * P_r+1 operator
    for j in range(1, N+1):
        
        if paddedBitStr[j-1] == '0' and paddedBitStr[j+1] == '0':
            copyBit[j] = flipMap[paddedBitStr[j]]
            flippedList.append(''.join(copyBit)[1:-1])
            copyBit = list(paddedBitStr)
        
    # adds row and column values for the sparse matrix
    for k in range(len(flippedList)):
        rowBare.append(basisMap[flippedList[k]])
        columnBare.append(i)
        
    #print(flippedList)
    flippedList.clear()

# -------------------------------
#
# create the sigma Z PXP hamiltonian perturbation
#
# -------------------------------

# print(basisList)
# print()

for i in range(basisLen):

    # add padding so that search doesnt go out of range
    paddedBitStr = basisList[i][-2] + basisList[i][-1] + basisList[i] + basisList[i][0] + basisList[i][1]
    copyBit = list(paddedBitStr)
    factor = 1

    # apply the PXP operator
    for j in range(2, N+2):
        
        if (paddedBitStr[j-1] == '0') and (paddedBitStr[j+1] == '0'):
            copyBit[j] = flipMap[paddedBitStr[j]]

            # apply sigmaZ_r-2 + sigmaZ_r+2
            r_neg2 = int(sigzMap[paddedBitStr[j-2]])
            r_pos2 = int(sigzMap[paddedBitStr[j+2]])
            factor = r_neg2 + r_pos2
            numList.append(factor)

            flippedList.append(''.join(copyBit)[2:-2])
            copyBit = list(paddedBitStr)
        
    # print(flippedList)

    # adds row and column values for the sparse matrix
    for k in range(len(flippedList)):
        rowFactor.append(basisMap[flippedList[k]])
        columnFactor.append(i)
        
    flippedList.clear()

# list of ones for the sparse matrix
onesList = np.ones(len(rowBare), dtype=int)

# create the sparse matrix and turn it into a Qobj
sparseBareHamiltonian = csr_matrix((onesList, (rowBare, columnBare)), shape=[basisLen, basisLen])
matrixBareHamiltonian = sparseBareHamiltonian.toarray()
#print(sparseBareHamiltonian)

sparseFactoredHamiltonian = csr_matrix((numList, (rowFactor, columnFactor)), shape=[basisLen, basisLen])
matrixFactoredHamiltonian = sparseFactoredHamiltonian.toarray()
print(numList)
print(sparseFactoredHamiltonian)

H0 = (ohms / 2 * sparseBareHamiltonian) + (-0.026 * ohms * sparseFactoredHamiltonian)

# print()
# print(H0)

H0 = qt.Qobj(H0)

# -------------------------------
#
# states and evolutions set ups
#
# -------------------------------

# diagonalize the sparse matrix
eigenvalues, eigenstates = H0.eigenstates()

# initial state
z2_str = z2_initial(N)
#print(z2_str)
#print(basisMap)
z2_index = basisMap[z2_str]
psi0 = qt.basis(basisLen, z2_index)

# time evolution of sparse matrix
tlist = np.linspace(0, 400, 500)
evolState = qt.sesolve(H0, psi0, tlist)

# -------------------------------
#
# create the driving hamiltonian
#
# -------------------------------

# create H1 operator for QobjEvo!
copyBasis = basisList
diagH1 = []

# switches 0s to -1s and keeps 1s the same for the copyBasis
# appends to diagH1 the dot product between each bit string and the 0 -> -1 Z2 state
# this is a diagonal perturbation and the diagonal elements are described above
for i in range(basisLen):

    bitString = list(copyBasis[i])
    #print(bitString)
    bitString = [int(i) for i in bitString]

    z2bitString = list(z2_initial(N))
    z2bitString = 2 * np.array([int(i) for i in z2bitString]) - 1
    #print(z2bitString)

    diagH1.append(np.dot(2 * np.array(bitString) - 1, z2bitString))

#print(diagH1)
# rows and columns lists for diagonal positions in H1
rowH1 = [i for i in range(basisLen)]
columnH1 = [i for i in range(basisLen)]

# creates sparse matrix with diagonals as diagH1 list
H1 = csr_matrix((diagH1, (rowH1, columnH1)), shape=[basisLen, basisLen])
#print(np.array(H1))
H1 = qt.Qobj(H1)

['0000', '0001', '0010', '0100', '0101', '1000', '1010']
[-2, -2, -2, -2, 2, -2, 2, -2, -2, 2, 2, 2, -2, 2, 2, 2]
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 16 stored elements and shape (7, 7)>
  Coords	Values
  (0, 1)	-2
  (0, 2)	-2
  (0, 3)	-2
  (0, 5)	-2
  (1, 0)	-2
  (1, 4)	2
  (2, 0)	-2
  (2, 6)	2
  (3, 0)	-2
  (3, 4)	2
  (4, 1)	2
  (4, 3)	2
  (5, 0)	-2
  (5, 6)	2
  (6, 2)	2
  (6, 5)	2
